<a href="https://colab.research.google.com/github/opoluliakh/data-analytics-learning/blob/main/Poluliakh_HW_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90_%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [7]:
import numpy as np
import math
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize


p1 = 0.19
p2 = 0.20
alpha = 0.05
power = 0.80


h = abs(proportion_effectsize(p1, p2))
print("Effect size (h):", h)


analysis = NormalIndPower()
n_per_group = analysis.solve_power(effect_size=h, alpha=alpha, power=power, alternative="larger")


n_per_group = float(np.asarray(n_per_group))
n_per_group_ceil = math.ceil(n_per_group)
total_n = 2 * n_per_group_ceil

print(f"n_per_group (raw) ≈ {n_per_group:.2f}")
print(f"Необхідна кількість користувачів у кожній групі: {n_per_group_ceil}")
print(f"Сумарна кількість користувачів: {total_n}")


Effect size (h): 0.025241594409087353
n_per_group (raw) ≈ 19407.28
Необхідна кількість користувачів у кожній групі: 19408
Сумарна кількість користувачів: 38816


Базове p₁ = 0.19, очікуване p₂ = 0.20  ефект = +0.01.  
При α = 0.05, power = 0.80 (one-sided): Cohen’s h ≈ 0.02524.  
Потрібно ≈ **19 408 користувачів у кожній групі**, разом ≈ **38 816** (округлено вгору).


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [8]:
import pandas as pd

df = pd.read_csv("/content/cookie_cats.csv")
df.head()

summary_7 = df.groupby("version")["retention_7"].agg(['sum','count','mean']) \
              .rename(columns={'sum':'successes','count':'n','mean':'rate'})
summary_7['rate_pct'] = (summary_7['rate']*100).round(2)

summary_7


,successes,n,rate,rate_pct
version,,,,
gate_30,8502,44700,0.190201,19.02
gate_40,8279,45489,0.182000,18.20


In [12]:
winner = summary_7['rate'].idxmax()
other = 'gate_40' if winner=='gate_30' else 'gate_30'
print(f"Вибірково вище retention_7 у: {winner}")

print("H0: p_gate_40 = p_gate_30")
print(f"Ha: p_{winner} > p_{other}  (правосторонній тест)")


Вибірково вище retention_7 у: gate_30
H0: p_gate_40 = p_gate_30
Ha: p_gate_30 > p_gate_40  (правосторонній тест)


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [14]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

tbl = df.groupby("version")["retention_7"].agg(["sum","count","mean"]).rename(
    columns={"sum":"successes","count":"n","mean":"rate"}
)
tbl


,successes,n,rate
version,,,
gate_30,8502,44700,0.190201
gate_40,8279,45489,0.182000


In [15]:
alpha = 0.05

control, treatment = "gate_30", "gate_40"

c_s, c_n, c_rate = tbl.loc[control, ["successes","n","rate"]]
t_s, t_n, t_rate = tbl.loc[treatment, ["successes","n","rate"]]

if t_rate > c_rate:
    alternative = "larger"
elif t_rate < c_rate:
    alternative = "smaller"
else:
    alternative = "two-sided"

z, p = proportions_ztest(count=[t_s, c_s], nobs=[t_n, c_n], alternative=alternative)

ci_c = proportion_confint(c_s, c_n, alpha=alpha, method="wilson")
ci_t = proportion_confint(t_s, t_n, alpha=alpha, method="wilson")

print(f"z statistic: {z:.4f}")
print(f"p-value: {p:.6f}")
print(f"Довірчий інтервал 95% для групи control ({control}): [{ci_c[0]:.4f}, {ci_c[1]:.4f}]")
print(f"Довірчий інтервал 95% для групи treatment ({treatment}): [{ci_t[0]:.4f}, {ci_t[1]:.4f}]")

better = treatment if t_rate > c_rate else control if c_rate > t_rate else "жодна"
sig = "Є статистично значуща різниця" if p < alpha else "Статистично значущої різниці не виявлено"
print(f"\n{sig} (α={alpha}). Краща вибірково: {better}. Δ = {t_rate - c_rate:.4f}")

overlap = not (ci_c[1] < ci_t[0] or ci_t[1] < ci_c[0])
print(f"CI overlap: {overlap}  (True = перетинаються)")


z statistic: -3.1644
p-value: 0.000777
Довірчий інтервал 95% для групи control (gate_30): [0.1866, 0.1939]
Довірчий інтервал 95% для групи treatment (gate_40): [0.1785, 0.1856]

Є статистично значуща різниця (α=0.05). Краща вибірково: gate_30. Δ = -0.0082
CI overlap: False  (True = перетинаються)


**Висновок:** На рівні значущості α = 0.05 різниця в `retention_7` між версіями **статистично значуща** (z = −3.1644, p = 0.000777): краща **control (gate_30)**, тоді як **treatment (gate_40)** має нижче утримання на ≈0.82 п.п. (Δ ≈ −0.0082). 95% довірчі інтервали **не перетинаються** (control: [0.1866; 0.1939], treatment: [0.1785; 0.1856]), що узгоджується з p-value і додатково вказує на реальну різницю між групами.


4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


**Гіпотези для χ²-тесту (α = 0.05):**  
- \(H_0\): версія гри та утримання на 7-й день **незалежні** (частки однакові в обох версіях).  
- \(H_a\): між версією гри та утриманням на 7-й день **є залежність** (частки відрізняються).


In [18]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

alpha = 0.05

ct = pd.crosstab(df["version"], df["retention_7"].astype(int))
print("Contingency table (version × retention_7):\n", ct, "\n")

chi2, p, dof, expected = chi2_contingency(ct, correction=False)
expected_df = pd.DataFrame(expected, index=ct.index, columns=ct.columns)

n = ct.to_numpy().sum()
cramers_v = np.sqrt(chi2 / n)

rates = (ct[1] / ct.sum(axis=1)).rename("rate")
better = rates.idxmax()
delta = float(rates.max() - rates.min())

print(f"chi2 = {chi2:.4f}, dof = {dof}, p-value = {p:.6f}")
print("Expected counts under H0:\n", expected_df.round(1), "\n")
print(f"Cramér's V = {cramers_v:.4f}")
print("Rates (retention_7):\n", rates.round(4))

conclusion = ("Відхиляємо H0 → є залежність між версією та утриманням"
              if p < alpha else
              "Не відхиляємо H0 → статистично значущої залежності не виявлено")
print(f"\nВисновок (α={alpha}): {conclusion}. "
      f"Вища частка утримання у {better}; Δ ≈ {delta:.4f}.")


Contingency table (version × retention_7):
 retention_7      0     1
version                 
gate_30      36198  8502
gate_40      37210  8279 

chi2 = 10.0132, dof = 1, p-value = 0.001554
Expected counts under H0:
 retention_7        0       1
version                     
gate_30      36382.9  8317.1
gate_40      37025.1  8463.9 

Cramér's V = 0.0105
Rates (retention_7):
 version
gate_30    0.1902
gate_40    0.1820
Name: rate, dtype: float64

Висновок (α=0.05): Відхиляємо H0 → є залежність між версією та утриманням. Вища частка утримання у gate_30; Δ ≈ 0.0082.


**χ²-тест (α=0.05):** χ² = 10.01, p = 0.0016 < 0.05 отже, **відхиляємо H₀** тому версія гри та утримання на 7-й день **залежні**.  
**Хто кращий:** **gate_30** — 19.02% vs 18.20% у gate_40 (Δ ≈ **+0.82 п.п.**).  
**Розмір ефекту:** Cramér’s V ≈ 0.0105, тобто
ефект **дуже малий**, хоч і статистично значущий (великий розмір вибірки).
